In [1]:
!python -V

Python 3.9.12


In [ ]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 73908)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

4.640808088756604

In [14]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():

    mlflow.set_tag("developer", "abideen")

    mlflow.log_param("train-data-path", "https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [16]:
# !pip install hyperopt xgboost

In [17]:
import xgboost as xgb

In [18]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [20]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=10,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [21]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[21:33:39] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:14.94780                          
[1]	validation-rmse:11.28701                          
[2]	validation-rmse:8.70862                           
[3]	validation-rmse:6.89891                           
[4]	validation-rmse:5.66299                           
[5]	validation-rmse:4.84707                           
[6]	validation-rmse:4.29951                           
[7]	validation-rmse:3.96065                           
[8]	validation-rmse:3.74140                           
[9]	validation-rmse:3.59390                           
[21:33:42] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:14.93938                                                    
[1]	validation-rmse:11.47824                                                    
[2]	validation-rmse:9.20348                       

In [41]:
mlflow.xgboost.autolog()

In [44]:
with mlflow.start_run():

    mlflow.set_tag("developer", "abideen")
    mlflow.log_param("train-data-path", "https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet")
    
    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[22:55:35] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:18.25785
[1]	validation-rmse:16.67126
[2]	validation-rmse:15.25114
[3]	validation-rmse:13.98169
[4]	validation-rmse:12.84948
[5]	validation-rmse:11.84129
[6]	validation-rmse:10.94755
[7]	validation-rmse:10.15635
[8]	validation-rmse:9.45913
[9]	validation-rmse:8.84562
[10]	validation-rmse:8.31004
[11]	validation-rmse:7.84276
[12]	validation-rmse:7.43566
[13]	validation-rmse:7.08319
[14]	validation-rmse:6.77981
[15]	validation-rmse:6.51786
[16]	validation-rmse:6.29346
[17]	validation-rmse:6.10184
[18]	validation-rmse:5.93870
[19]	validation-rmse:5.80065
[20]	validation-rmse:5.68228
[21]	validation-rmse:5.58328
[22]	validation-rmse:5.49884
[23]	validation-rmse:5.42666
[24]	validation-rmse:5.36437
[25]	validation-rmse:5.31123
[26]	validation-rmse:5.26536
[27]	validation-rmse:5.22632
[28]	validation-rmse:5.19220
[29]	validation-rmse:5.16393
[30]	validati

2022/05/29 22:57:33 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID c458c2f0cd6f498899439f5aff540282. Failed operations: [MlflowException("Changing param values is not allowed. Params were already logged=\'[{\'key\': \'num_boost_round\', \'old_value\': None, \'new_value\': \'1000\'}, {\'key\': \'early_stopping_rounds\', \'old_value\': None, \'new_value\': \'50\'}, {\'key\': \'maximize\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'verbose_eval\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'custom_metric\', \'old_value\': None, \'new_value\': \'None\'}]\' for run ID=\'c458c2f0cd6f498899439f5aff540282\'.")]')]


In [48]:
# Predict on a Pandas DataFrame:
import mlflow
logged_model = 'runs:/c458c2f0cd6f498899439f5aff540282/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

[01:04:18] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [49]:
logged_model

'runs:/c458c2f0cd6f498899439f5aff540282/models_mlflow'

In [50]:
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

[01:04:25] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [51]:
y_pred[:10]

array([ 6.85117627, 14.01973656,  7.14731293, 11.48109148,  4.70549955,
       32.92775749, 17.43915017, 24.45530892, 13.33550679, 21.81090745])

In [52]:
y_pre = xgboost_model.predict(valid)
y_pre[:10]

array([ 4.912515 , 12.481487 ,  6.5756993, 11.670506 ,  4.027239 ,
       32.88778  , 17.413437 , 15.329085 , 13.348291 , 19.535524 ],
      dtype=float32)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

mlflow.sklearn.autolog()

with mlflow.start_run():
    mlflow.set_tag("developer", "abideen")
    mlflow.log_param("train-data-path", "https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet")
    
    mlmodel = GradientBoostingRegressor()
    mlmodel.fit(X_train, y_train)

    y_pred = mlmodel.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.sklearn.log_model(mlmodel, artifact_path="models_mlflow")